In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import random
import networkx as nx

In [ ]:
# Create network and startsituation
def start(nodes, sickPerc, radius):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Situation.append(1)
        else:
            Situation.append(0)
    return Network, Situation

def startcolor(nodes, sickPerc, radius):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    Colorlist = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Situation.append(1)
            Colorlist.append('tomato')
        else:
            Situation.append(0)
            Colorlist.append('palegreen')
    return Network, Situation, Colorlist

In [ ]:
## SI
# 0 for S, 1 for I
def updateSI(y, beta, Network):
    yNew = []
    for i in range(len(Start)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                Phealthy *= (1 - beta*y[j])
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
            else:
                yNew.append(y[i])
        else:
            yNew.append(y[i])
    return yNew

def spreaddiseaseSI(NLoops, StartSituation, beta, Network):
    yList = [StartSituation]
    yNew = StartSituation
    for i in range(NLoops):
        y = yNew
        yNew = updateSI(y, beta, Network)
        yList.append(yNew)
    return yList    

In [ ]:
## SI with color
# 0 for S, 1 for I
def updateSIcolor(y, beta, Network):
    yNew = []
    Colorlist = []
    for i in range(len(y)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                Phealthy *= (1 - beta*y[j])
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
                Colorlist.append('tomato')
            else:
                yNew.append(y[i])
                Colorlist.append('palegreen')
        else:
            yNew.append(y[i])
            Colorlist.append('tomato')
    return yNew, Colorlist

def spreaddiseaseSIcolor(NLoops, StartSituation, beta, Network):
    yNew = StartSituation
    pos = nx.get_node_attributes(Network, 'pos')
    for i in range(NLoops):
        y = yNew
        yNew, Colorlist = updateSIcolor(y, beta, Network)
        plt.figure(figsize = (8, 8))
        nx.draw_networkx_edges(Network, pos, alpha= 0.4)
        nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
        plt.axis('off')
        plt.show()

def everythingSIcolor(nodes, sickPerc, radius, NLoops, beta):
    Network, Situation, Colorlist = startcolor(nodes, sickPerc, radius)
    
    pos = nx.get_node_attributes(Network, 'pos')
    plt.figure(figsize = (8, 8))
    nx.draw_networkx_edges(Network, pos, alpha= 0.4)
    nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
    plt.axis('off')
    plt.show()
    
    spreaddiseaseSIcolor(NLoops, Situation, beta, Network)

In [ ]:
## SIS
# 0 for S, 1 for I
def updateSIS(y, beta, gamma, Network):
    yNew = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
            else:
                yNew.append(y[i])
        else:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(0)
            else:
                yNew.append(y[i])
    return yNew

def spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, Network):
    yList = [StartSituation]
    yNew = StartSituation
    for i in range(NLoops):
        y = yNew
        yNew = updateSIS(y, beta, gamma, Network)
        yList.append(yNew)
    return yList

def countingSIS(situations):
    S = []
    I = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 2)[0])
        I.append(np.bincount(situations[i], minlength = 2)[1])
    return S, I

def everythingSIS(nodes, sickPerc, radius, NLoops, beta, gamma):
    G, StartSituation = start(nodes, sickPerc, radius)
    situations = spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, G)
    S, I = countingSIS(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S')
    plt.plot(time, I, label = 'I')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return G, S, I

def averageSIS(nodes, sickPerc, radius, NLoops, beta, gamma, N):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    for i in range(N):
        G, StartSituation = start(nodes, sickPerc, radius)
        situations = spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, G)
        S, I = countingSIS(situations)
        AverageS += S
        AverageI += I
    AverageS = AverageS/N
    AverageI = AverageI/N
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S')
    plt.plot(time, AverageI, label = 'I')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()
    return AverageS, AverageI

In [ ]:
## SIR
# 0 for S, 1 for I, 2 for R
def updateSIR(y, beta, gamma, Network):
    yNew = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
            else:
                yNew.append(y[i])
        elif y[i] == 1:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(2)
            else:
                yNew.append(y[i])
        else:
            yNew.append(y[i])
    return yNew

def spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, Network):
    yList = [StartSituation]
    yNew = StartSituation
    for i in range(NLoops):
        y = yNew
        yNew = updateSIR(y, beta, gamma, Network)
        yList.append(yNew)
    return yList

def countingSIR(situations):
    S = []
    I = []
    R = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 3)[0])
        I.append(np.bincount(situations[i], minlength = 3)[1])
        R.append(np.bincount(situations[i], minlength = 3)[2])
    return S, I, R

def everythingSIR(nodes, sickPerc, radius, NLoops, beta, gamma):
    G, StartSituation = start(nodes, sickPerc, radius)
    situations = spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, G)
    S, I, R = countingSIR(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S')
    plt.plot(time, I, label = 'I')
    plt.plot(time, R, label = 'R')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return G, S, I, R

def averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    AverageR = np.zeros(NLoops + 1)
    for i in range(N):
        G, StartSituation = start(nodes, sickPerc, radius)
        situations = spreaddiseaseSIR(NLoops, StartSituation, beta, gamma, G)
        S, I, R = countingSIR(situations)
        AverageS += S
        AverageI += I
        AverageR += R
    AverageS = AverageS/N
    AverageI = AverageI/N
    AverageR = AverageR/N
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S')
    plt.plot(time, AverageI, label = 'I')
    plt.plot(time, AverageR, label = 'R')

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12, loc = 'upper right')
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()
    return AverageS, AverageI, AverageR

In [ ]:
## SI testing
# People connected to i'th person (0 to 10)
Network = [[2], [2], [0, 1, 4], [4], [2, 3, 6], [6], [4, 5, 7, 10], [6, 8], [7, 9], [8, 10], [6, 9]]
# 1 is sick, 0 is healthy
Start = [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
beta = 0.5
NLoops = 10

yNew = Start
for i in range(NLoops):
    y = yNew
    yNew = updateSI(y, beta, Network)
    print(yNew)

In [ ]:
## SIR testing
Network = [[2], [2], [0, 1, 4], [4], [2, 3, 6], [6], [4, 5, 7, 10], [6, 8], [7, 9], [8, 10], [6, 9]]
Start = [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
beta = 0.5
gamma = 0.2
NLoops = 10

print(spreaddiseaseSIR(NLoops, Start, beta, gamma, Network))

In [ ]:
# 1 network for SIR
nodes = 100
sickPerc = 0.1
radius = 0.1
beta = 0.2
gamma = 0.02
NLoops = 200

G, S, I, R = everythingSIR(nodes, sickPerc, radius, NLoops, beta, gamma)

In [ ]:
# Average over N networks for SIR
N = 100
nodes = 100
sickPerc = 0.1
radius = 0.1
beta = 0.2
gamma = 0.02
NLoops = 200

AverageS, AverageI, AverageR = averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N)

In [ ]:
# 1 network for SIS
N = 100
nodes = 100
sickPerc = 0.1
radius = 0.1
beta = 0.2
gamma = 0.02
NLoops = 200

G, S, I = everythingSIS(nodes, sickPerc, radius, NLoops, beta, gamma)

In [ ]:
# Average over N networks for SIS
N = 100
nodes = 100
sickPerc = 0.1
radius = 0.1
beta = 0.2
gamma = 0.02
NLoops = 200

AverageS, AverageI = averageSIS(nodes, sickPerc, radius, NLoops, beta, gamma, N)

In [ ]:
nodes = 200
sickPerc = 0.1
radius = 0.12
beta = 0.3
gamma = 0.02
NLoops = 5

everythingSIcolor(nodes, sickPerc, radius, NLoops, beta)

In [ ]:
# State in network test
'''def start(nodes, sickPerc, radius):
    Network = nx.random_geometric_graph(nodes, radius)
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Network.nodes[i]['State'] = 1
        else:
            Network.nodes[i]['State'] = 0
    return Network'''